In [1]:
import numpy as np
import tensorflow as tf
from keras import layers
from keras.layers import Input, Add, Dense, Dropout,Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.callbacks import EarlyStopping
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import display
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from keras.applications import VGG19
from keras.constraints import maxnorm
from keras import optimizers
from keras.optimizers import Adam, SGD , RMSprop
import scipy.misc
from matplotlib.pyplot import imshow
import matplotlib.image as mpimg
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from PIL import Image

import keras.backend as K
#K.set_image_data_format('channels_last')
#K.set_learning_phase(1)

Using TensorFlow backend.


In [0]:
num_px = 128
C = 2
B_size = 64
E = 20

In [5]:
base_model = VGG19(weights = "imagenet", include_top=False, 
                   input_shape = (num_px, num_px, 3))
x = base_model.output
x = Flatten()(x)
x = Dense(1024, activation = "relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
x = Dense(256, activation = "relu")(x)
x = Dropout(0.5)(x)
x = BatchNormalization()(x)
preds = Dense(C, activation = "softmax")(x)

model = Model(input = base_model.input, output = preds)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  del sys.path[0]


In [0]:
for layer in model.layers[:16]:
    layer.trainable=False
for layer in model.layers[16:]:
    layer.trainable=True

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [0]:
early = EarlyStopping(monitor = 'val_acc', min_delta = 0, 
                      patience = 10, verbose= 1 , mode = 'auto')

In [9]:
model.compile(loss = "binary_crossentropy", 
              optimizer = SGD(lr=0.01, momentum=0.9, decay=1e-6, nesterov=True), 
              metrics=["accuracy"])

W0731 17:40:57.039293 139883695503232 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0731 17:40:57.049780 139883695503232 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
trainAug = ImageDataGenerator(rescale = 1./255, fill_mode = "nearest")

trainGen = trainAug.flow_from_directory(
                    '/content/chest_xray/train',
                    target_size = (num_px, num_px),
                    batch_size = B_size,
                    shuffle = True,
                    class_mode = "categorical")

Found 5216 images belonging to 2 classes.


In [11]:
testGen = trainAug.flow_from_directory(
                    '/content/chest_xray/test',
                    target_size = (num_px, num_px),
                    batch_size = B_size,
                    shuffle = False,
                    class_mode = "categorical")

Found 624 images belonging to 2 classes.


In [12]:
ValGen = trainAug.flow_from_directory(
                    '/content/chest_xray/val',
                    target_size = (num_px, num_px),
                    batch_size = B_size,
                    shuffle = False,
                    class_mode = "categorical")

Found 16 images belonging to 2 classes.


In [13]:
H = model.fit_generator(trainGen, epochs=E, validation_data = ValGen, validation_steps = 16/64,steps_per_epoch = 5216//64, callbacks = [early])

Epoch 1/20
81/81 [==============================] - 54s 670ms/step - loss: 0.3728 - acc: 0.8314 - val_loss: 3.7444 - val_acc: 0.5000
Epoch 2/20
81/81 [==============================] - 50s 616ms/step - loss: 0.1061 - acc: 0.9624 - val_loss: 2.4104 - val_acc: 0.5000
Epoch 3/20
81/81 [==============================] - 50s 617ms/step - loss: 0.0744 - acc: 0.9755 - val_loss: 2.5998 - val_acc: 0.5000
Epoch 4/20
81/81 [==============================] - 51s 625ms/step - loss: 0.0627 - acc: 0.9782 - val_loss: 1.6636 - val_acc: 0.5000
Epoch 5/20
81/81 [==============================] - 51s 626ms/step - loss: 0.0464 - acc: 0.9853 - val_loss: 2.1147 - val_acc: 0.5000
Epoch 6/20
81/81 [==============================] - 51s 626ms/step - loss: 0.0466 - acc: 0.9828 - val_loss: 1.1676 - val_acc: 0.6250
Epoch 7/20
81/81 [==============================] - 52s 639ms/step - loss: 0.0368 - acc: 0.9877 - val_loss: 0.0362 - val_acc: 1.0000
Epoch 8/20
81/81 [==============================] - 50s 621ms/step - 

In [0]:
model.save('model.h5')

In [14]:
testGen.reset()
predicts = model.predict_generator(testGen, steps = (624 // B_size) + 1) 
predicts = np.argmax(predicts, axis=1)

print("Test set accuracy: " + 
      str(accuracy_score(testGen.classes, predicts, normalize=True) * 100) 
      + "%") 

Test set accuracy: 90.06410256410257%
